In [1]:
#Importing libraries
import torch
import torchvision
import torch.nn as nn
import torch.utils.data as data
import numpy as np
from PIL import Image
from torch.autograd import Variable
from torchvision import models, transforms
import glob
from PIL import Image

use_gpu = torch.cuda.is_available()

In [45]:
#Loading list of images from Coco Dataset
images = glob.glob('/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/*.jpg')
len(images)

40504

In [5]:


class FeatureExtractor(nn.Module):
    def __init__(self, submodule, extracted_layers):
        super(FeatureExtractor, self).__init__()
        self.submodule = submodule
        self.extracted_layers = extracted_layers

    def forward(self, x):
        outputs = []
        for name, module in self.submodule._modules.items():
            if name is "fc": x = x.view(x.size(0), -1)
            x = module(x)
            if name in self.extracted_layers:
                outputs.append(x)
        return outputs

extract_list = ["conv1", "maxpool", "layer1", "avgpool", "fc"]

#Loading pretrained Resnet Model
resnet = models.resnet50(pretrained=True)

<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_146554/3281886448.py:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if name is "fc": x = x.view(x.size(0), -1)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


In [ ]:
#Transforms
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()]
)

In [ ]:
vec = []
extract_result = FeatureExtractor(resnet, extract_list)
for num, i in enumerate(images):
    if num%1000==0:
        print(num)
    img = Image.open(i)
    img = transform(img)
#     print(img.shape)
    if(img.shape[0]==1):
        img = torch.cat((img, img, img), dim=0)
    x = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)
    x = x.cuda()
    resnet = resnet.cuda()
    vec.append(extract_result(x)[4].detach().cpu())  # [0]:conv1  [1]:maxpool  [2]:layer1  [3]:avgpool  [4]:fc

In [54]:
#Saving the computed embeddings
import pickle

# write list to binary file
def write_list(a_list):
    # store list in binary file so 'wb' mode
    with open('/home/bsantra/divyanshu/graphml/Sub-GC/data/embeddings', 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')

write_list(vec)

Done writing list into a binary file


In [12]:
vec = np.array(vec)
vec = vec.reshape(40504)

In [33]:
tensors_new = vec
tensors = [t.numpy() for t in tensors_new]

In [30]:
import pickle

tensors_new = np.array(pickle.load( open( '/home/bsantra/divyanshu/graphml/Sub-GC/data/embeddings', 'rb')))

/tmp/ipykernel_146554/2755872080.py:3: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  tensors_new = np.array(pickle.load( open( '/home/bsantra/divyanshu/graphml/Sub-GC/data/embeddings', 'rb')))
/tmp/ipykernel_146554/2755872080.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensors_new = np.array(pickle.load( open( '/home/bsantra/divyanshu/graphml/Sub-GC/data/emb

In [36]:
tensors = np.array(tensors)

In [38]:
print(tensors.shape)
tensors = tensors.reshape((40504, 1000))

(40504, 1, 1000)


In [40]:
from sklearn.cluster import KMeans
import numpy as np
kmeans = KMeans(n_clusters=500, random_state=0).fit(tensors)
kmeans.labels_

kmeans.cluster_centers_

array([[-1.0806928 , -0.32905596, -0.5152335 , ..., -0.51872766,
         1.0767077 ,  0.9018179 ],
       [-1.081657  , -0.34023595, -0.58720237, ..., -0.56577164,
         1.1202066 ,  0.82615304],
       [-1.116573  , -0.376202  , -0.56048334, ..., -0.65315837,
         1.0868918 ,  0.9554055 ],
       ...,
       [-1.0760316 , -0.31466755, -0.5760912 , ..., -0.5871613 ,
         1.1188657 ,  0.9247462 ],
       [-1.1848181 , -0.3474851 , -0.5529079 , ..., -0.69459575,
         1.0823302 ,  1.050059  ],
       [-1.0307306 , -0.3128616 , -0.5287717 , ..., -0.53360856,
         1.0906963 ,  0.82646185]], dtype=float32)

In [47]:
predictions = kmeans.predict(tensors)

In [50]:
clusters = {}
for i in range(len(predictions)):
    if str(predictions[i]) not in clusters.keys():
        clusters[str(predictions[i])] = []
    clusters[str(predictions[i])].append(images[i])

In [53]:
for i in clusters:
    print(len(clusters[i]))

132
94
117
111
82
89
98
79
65
96
114
78
105
73
79
105
124
88
145
99
107
136
94
85
89
106
90
96
143
65
116
82
86
71
60
124
87
76
106
78
106
67
100
70
67
76
110
88
111
67
114
89
111
51
76
96
90
153
86
92
132
115
98
85
122
111
62
129
95
111
53
92
27
63
94
85
110
55
79
111
156
111
59
90
42
85
82
101
64
92
120
40
61
67
103
148
31
109
87
61
107
103
75
92
109
73
39
105
77
69
68
134
88
83
105
109
52
59
113
119
118
84
131
103
108
115
89
83
147
54
93
105
97
82
126
95
58
73
86
116
127
127
77
81
18
81
46
75
63
63
89
140
85
91
110
76
92
134
53
74
65
66
71
119
98
124
81
96
128
85
105
98
65
118
67
75
71
87
96
69
88
77
62
87
95
93
122
85
106
116
90
99
60
38
110
120
78
102
92
62
75
111
83
117
115
38
57
103
104
80
53
74
97
47
85
95
67
56
72
86
74
131
86
29
79
79
51
84
47
97
80
51
46
84
70
85
108
94
38
59
79
111
55
114
50
91
96
107
81
126
87
131
72
66
109
67
70
152
88
57
134
90
88
35
111
42
68
70
94
93
71
84
52
39
37
52
72
115
105
87
65
40
103
135
64
48
60
95
117
81
58
81
80
118
70
61
89
52
91
74
108
59


In [64]:
clusters

{'222': ['/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000230268.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000164830.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000417481.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000156533.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000435707.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000552656.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000397375.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000370478.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000505386.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000045655.jpg',
  '/home/bsantra/divyanshu/graphml/Sub-GC/data/val2014/COCO_val2014_000000188509.jpg',
  '/home/bsantra/divyanshu/graphml/S

In [65]:
import json
with open("/home/bsantra/divyanshu/graphml/Sub-GC/data/clusters.json", "w") as outfile:
    json.dump(clusters, outfile)